<a href="https://colab.research.google.com/github/haydenclose/Cloud_based_Oil_Detection/blob/main/Cloud_Based_Oil_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloud Based Analysis of Oil Spills from Wrecks
This is a workflow on how to use Google Earth Engine (GEE) with Google Colab to identify and detect oil from shipwreck

Created By Hayden Close


#1 Installation and set-up of workspace

##1.2 Install and import required libraries
install via `!pip install name` and
`import name`

use `> /dev/null 2>&1; ` to supress outputs as lengthy and clog up the screen

In [ ]:
def Install_Packages():
  !pip install geemap > /dev/null 2>&1;                                                               # Need to install geeMap for easy iteractive plotting
  !pip install geopandas > /dev/null 2>&1;                                                            # Install geopandas
  !pip install plotly > /dev/null 2>&1;                                                               # Install Ploty for interactive graphs
  import ee                                                                                           # To access the earth engine catalogue
  import geemap                                                                                       # Imports the geeMap package into environment
  import geopandas                                                                                    # Makes working with geospatial data in python easier
  import pandas as pd                                                                                 # Useful package to read/export csv's etc...
  import ipyleaflet                                                                                   # Needed for interactive plotting
  import ipywidgets as widgets                                                                        # Way to have widgets onto geemaps
  import math                                                                                         # Used in the gammafilter function
  import plotly.express as px                                                                         # Used to plot the spills over time or the values from a line
  return print('Libraries installed')

Libraries installed


##2.1  GammaMap Speckle filter

This code defines three functions: `powerToDb`, `dbToPower`, and `gammaMap`. These functions are used to perform gamma map filtering on a Sentinel-1 image to reduce speckle noise.

The `powerToDb` function takes an image in power scale as input and converts it to decibel (dB) scale using the formula `10 * log10(img)`. The `dbToPower` function performs the inverse operation, converting an image in dB scale back to power scale using the formula `10^(img/10)`.

The `gammaMap` function takes an image in dB scale as input and performs gamma map filtering on it to reduce speckle noise. The function first converts the image from dB to power scale using the `dbToPower` function. It then calculates the mean and variance of the image within a square kernel of size `ksize` using the `reduceNeighborhood` method. The kernel is defined using an ee.Kernel.fixed object with weights set to 1.

The function then calculates a “pure speckle” threshold based on the equivalent number of looks (`enl`) parameter. If the coefficient of variation (`ci`) within the kernel is less than or equal to this threshold (`cu`), the function returns the simple mean of the pixel values within the kernel.

If `ci` is greater than `cu` but less than a maximum value (`cmax`), the function calculates a filtered value for each pixel using a refined Lee filter. This filtered value is calculated using a complex formula that involves several intermediate variables (`alpha`, `b`, `d`, and `f`). The final result is obtained by dividing `b * mean + sqrt(d)` by `alpha * 2`.

If `ci` is greater than or equal to `cmax`, the function does not filter the pixel value at all. Instead, it returns the original pixel value from the input image.

The final result is obtained by combining these three cases into a single image using the ee.`ImageCollection.reduce` method with the `ee.Reducer.firstNonNull` reducer. This produces an image where pixels with low speckle noise are replaced by their mean value, pixels with moderate speckle noise are replaced by their filtered value, and pixels with high speckle noise are left unchanged.

The code is quite complex so not fully annotated below but more information can be found from the source of the code [Mygeoblog.com](https://mygeoblog.com/2021/01/21/sentinel-1-speckle-filter-gamma-map/)


In [ ]:
# Implementation by Andreas Vollrath (ESA), inspired by Johannes Reiche (Wageningen)
def powerToDb(img):
  return ee.Image(10).multiply(img.log10())
def dbToPower(img):
  return ee.Image(10).pow(img.divide(10))
def gammaMap(img):
  ksize = 7
  enl = 5
  bandNames = img.bandNames()

  # Convert image from dB to natural values
  nat_img = dbToPower(img)

  # Square kernel, ksize should be odd (typically 3, 5 or 7)
  weights = ee.List.repeat(ee.List.repeat(1,ksize),ksize)

  # ~~(ksize/2) does integer division in JavaScript
  kernel = ee.Kernel.fixed(ksize,ksize, weights, math.floor(ksize/2), math.floor(ksize/2), False)

  # Get mean and variance
  mean = nat_img.reduceNeighborhood(ee.Reducer.mean(), kernel)
  variance = nat_img.reduceNeighborhood(ee.Reducer.variance(), kernel)

  # "Pure speckle" threshold
  ci = variance.sqrt().divide(mean);  # square root of inverse of enl

  # If ci <= cu, the kernel lies in a "pure speckle" area -> return simple mean
  cu = 1.0/math.sqrt(enl)

  # If cu < ci < cmax the kernel lies in the low textured speckle area -> return the filtered value
  cmax = math.sqrt(2.0) * cu

  alpha = ee.Image(1.0 + cu*cu).divide(ci.multiply(ci).subtract(cu*cu))
  b = alpha.subtract(enl + 1.0)
  d = mean.multiply(mean).multiply(b).multiply(b).add(alpha.multiply(mean).multiply(nat_img).multiply(4.0*enl))
  f = b.multiply(mean).add(d.sqrt()).divide(alpha.multiply(2.0))

  caster = ee.Dictionary.fromLists(bandNames,ee.List.repeat('float',3))
  img1 = powerToDb(mean.updateMask(ci.lte(cu))).rename(bandNames).cast(caster)
  img2 = powerToDb(f.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames).cast(caster)
  img3 = img.updateMask(ci.gte(cmax)).rename(bandNames).cast(caster)

  # If ci > cmax do not filter at all (i.e. we don't do anything, other then masking)
  result = ee.ImageCollection([img1,img2,img3]) \
    .reduce(ee.Reducer.firstNonNull()).rename(bandNames)

  # Compose a 3 band image with the mean filtered "pure speckle", the "low textured" filtered and the unfiltered portions
  return result


## 2.2 Functions to display the map
The first function `addDate` uses the `set()` method to set a property in the polygons which are stored as Earth Engine Objects. In this case we want to add the date and colour number.

The section function doess... copy alot of the info from previous script to describe

In [ ]:
def addDate(feature):                                                                               # Function to add date to the individual polygons
  return feature.set({'Date': NxtDate,'Colour': n})                                                 # Adds the date to the feature part of each polygon in the feature collection

def Sentinel1_OilMapping():                                                                         # Function to display map
 Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                           # Load in the wreck list
 Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Latitude'])               # Get the latitude of selected wreck
 Lon = Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Longitude']                             # Get the longitude of selected wreck
 geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                 # Loction of chosen wreck

  # Get the image collection
 ImgCol = (ee.ImageCollection('COPERNICUS/S1_GRD').                                                 # Selects the Sentinel 1 image collection
    filterDate(str(StartDate.value), str(EndDate.value)).                                           # Selects only the dates from time period chosen above
    filterMetadata('instrumentMode', 'equals', 'IW').                                               # Selects the instrument mode that we want
    filter(ee.Filter.eq('orbitProperties_pass', str(orbit.value))).                                 # Selects the orbit path we want
    filterBounds(geom))                                                                             # Selects only images that our wreck is contained within
 global ImgListRaw, Oil_Polygons, Datatable, img, n, NxtDate                                        # Need to make sure it brings the data from outside the function
 ImgCol_gammaMap = ImgCol.map(gammaMap)                                                             # Process the ImageCollection through the gammaMap algorithm
 ImgListGamma = ee.ImageCollection(ImgCol_gammaMap).toList(99999)                                   # Creates a list of the GAMMA corrected images to select from
 ImgListRaw = ee.ImageCollection(ImgCol).toList(99999)                                              # Creates a list of the RAW images to select from
 Oil_Polygons = ee.FeatureCollection([])                                                            # Need an empty FeatureCollection to Append our polygons to, add here so resets
 Datatable = pd.DataFrame(columns = ['Wreck_Name', 'Date', 'Oil_Area_m2','Threshold'])              # Empty dataframe for polygon oil spill area
 MultiMap = geemap.Map()                                                                            # Base map
 MultiMap.centerObject(geom, 10)                                                                    # Center the map on the wreck
 MultiMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                   # Add wreck locations

 n = 0                                                                                              # used to add or subtract to change the image
 img = ee.Image(ee.List(ImgListRaw).get(n)).select('VV')                                            # Gets the first image to display
 year = ee.Image(img).getString('system:index').slice(-50, 21)                                      # Select out the year
 month = ee.Image(img).getString('system:index').slice(-46, 23)                                     # Select out the month
 day = ee.Image(img).getString('system:index').slice(-44, 25)                                       # Select out the day
 NxtDate = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                    # Format to date format e.g. 2023_01_01
 NxtDate = NxtDate.getInfo()                                                                        # ee.String an object on server side so need to extract it
 img_params = {'bands':'VV', 'min':-25, 'max':5}                                                    # Display setting for the VV band
 MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                         # Add the image to the map
 output_widget = widgets.Output(layout={'border': '1px solid black'})                               # Set up widget for adding date
 output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')            # Method to add widget
 MultiMap.add_control(output_control)                                                               # Adds the widget

 with output_widget:                                                                                # The date widget update
    print(NxtDate)                                                                                  # Prints the first image date
#################################################################################
 DetectOilButton = widgets.Button(description="Detect potential oil")                               # Button widget to add to the map
 DetectOilButtonOutput = widgets.Output()                                                           # Output to display for the widget
 DetectOilButtonWidget = ipyleaflet.WidgetControl(widget=DetectOilButton, position='bottomright')   # Method to add to the map
 MultiMap.add_control(DetectOilButtonWidget)                                                        # Adds to the map

 ChangeThresh =  widgets.FloatSlider(                                                               # Defines the Oil threshold slider
     value=-23.5,                                                                                   # Default
     min=-30.0,                                                                                     # Min value
     max=-15.0,                                                                                     # Max value
     step=0.5,                                                                                      # interval step value
     description='Threshold (default =-25):',                                                       # Description
     orientation='horizontal',                                                                      # Horizontal or Vertical
     readout=True,                                                                                  # Prints value
     readout_format='.1f',)                                                                         # Format of value (1 decimal here)
 ChangeThreshOutput = widgets.Output()                                                              # Output to display for the widget
 ChangeThreshWidget = ipyleaflet.WidgetControl(widget=ChangeThresh, position='bottomleft')          # Method to add to the map
 MultiMap.add_control(ChangeThreshWidget)                                                           # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
   global Oil_Polygonstmp, n                                                                        # Need to make sure it brings the data from outside the function
   with DetectOilButtonOutput:                                                                      # Below happens when button clicked
    img = ee.Image(ee.List(ImgListGamma).get(n)).select('VV')                                       # Want to select based on the gammma image
    AoI = ee.FeatureCollection(MultiMap.draw_features)                                              # Extracts the drawn polygon from the map
    with ChangeThreshOutput:                                                                        # Allows the threshold to change
      Oil = (img.lt(ChangeThresh.value).selfMask().rename('Pixels'))                                # Selects pixel values less than -23.5
    MultiMap.addLayer(Oil, {'palette': 'FF0000'}, ('Pixels'))                                       # Displays those with pixels value 23.5
    Oil_Polygonstmp = Oil.reduceToVectors(geometry = AoI,                                           # Polygon extracted from the map
                             scale = 10,                                                            # Resolution of data in meters
                             geometryInNativeProjection =False,                                      # Use the image projection
                             maxPixels = 1e10,                                                      # Max pixels GEE will deal with
                             eightConnected =True)                                                  # Polgons will count as connect if at a diagonal
    Oil_Polygonstmp = Oil_Polygonstmp.filter(ee.Filter.gt('count',25))                              # Filters the polgons whic have more than 25 pixels
    Oil_Polygonstmp = Oil_Polygonstmp.map(addDate)                                                  # Adds image date to the polygons
    empty = ee.Image().byte()
    outline = empty.paint(featureCollection= Oil_Polygonstmp,
              color= 'Colour')
    MultiMap.addLayer(outline.randomVisualizer(),name ='Temp Oil Spill Polygons')

 DetectOilButton.on_click(on_button_clicked)                                                        # Makes changes when clicked
########################################################################################################
 NextButton = widgets.Button(description="Next image")                                              # Button widget to add to the map
 NextButtonOutput = widgets.Output()                                                                # Output to display for the widget
 NextButtonWidget = ipyleaflet.WidgetControl(widget=NextButton, position='bottomright')             # Method to add to the map
 MultiMap.add_control(NextButtonWidget)                                                             # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
   global n, NxtDate, img, ImgListRaw, Datatable, Oil_Polygons, Oil_Polygonstmp                     # Need this as otherwise it looks internally of the function for values
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   empty = ee.Image().byte()                                                                        # Empty image
   outline = empty.paint(featureCollection= Oil_Polygons, color= 'Colour')                          # Add the polygon and colour
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value], 'Date':[NxtDate],'Oil_Area_m2': [spillarea],'Threshold':[ChangeThresh.value]})# Combine the data together

   if NxtDate in Datatable['Date'].values:                                                          # Check if the date already exists in the DataFrame
    # Overwrite the existing row
    Datatable.loc[Datatable['Date'] == NxtDate, ['Wreck_Name', 'Date', 'Oil_Area_m2','Threshold']] = [WRKdropdown.value, NxtDate, spillarea, ChangeThresh.value  ]
   else:
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n += 1                                                                                           # Add one to the number in the image list
   with NextButtonOutput:                                                                           # Below happens when button clicked
    # The metadata doesnt have the date, have to extract from image name (called index)
     img = ee.Image(ee.List(ImgListRaw).get(n)).select('VV')                                        # Selects the next image in the list
     year = ee.Image(img).getString('system:index').slice(-50, 21)                                  # Select out the year
     month = ee.Image(img).getString('system:index').slice(-46, 23)                                 # Select out the month
     day = ee.Image(img).getString('system:index').slice(-44, 25)                                   # Select out the day
     NxtDate = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                # Format to date format e.g. 2023_01_01
     NxtDate = NxtDate.getInfo()                                                                    # ee.String an object on server side so need to extract it
     with output_widget:                                                                            # Method to update date widget
      print(NxtDate)                                                                                # Updates the image date widget
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     empty = ee.Image().byte()                                                                      # Empty image
     outline = empty.paint(featureCollection= Oil_Polygons,color= 'Colour')                         # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map

 NextButton.on_click(on_button_clicked)                                                             # actions the function when the button clicked
############################################################################################################
 PrevButton = widgets.Button(description="Previous image")                                          # Button widget to add to the map
 PrevButtonOutput = widgets.Output()                                                                # Output to display for the widget
 PrevButtonWidget = ipyleaflet.WidgetControl(widget=PrevButton, position='bottomleft')              # Method to add to the map
 MultiMap.add_control(PrevButtonWidget)                                                             # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
  ## Operations that need to happen before the button action
   global n, NxtDate, img, ImgListRaw, Datatable, Oil_Polygons, Oil_Polygonstmp                     # Need this as otherwise it looks internally of the function for values
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   empty = ee.Image().byte()                                                                        # Empty image
   outline = empty.paint(featureCollection= Oil_Polygons, color= 'Colour')                          # Add the polygon and colour
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value], 'Date':[NxtDate],'Oil_Area_m2': [spillarea],'Threshold':[ChangeThresh.value]})# Combine the data together

   if NxtDate in Datatable['Date'].values:                                                          # Check if the date already exists in the DataFrame
    # Overwrite the existing row
    Datatable.loc[Datatable['Date'] == NxtDate, ['Wreck_Name', 'Date', 'Oil_Area_m2','Threshold']] = [WRKdropdown.value, NxtDate, spillarea, ChangeThresh.value  ]
   else:
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n -= 1                                                                                           # Add one to the number in the image list
   output_widget.clear_output()                                                                     # Remove the widget of the image date
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   with PrevButtonOutput:                                                                           # Below happens when button clicked
# The metadata doesnt have the date, have to extract from image name (called index)
     img = ee.Image(ee.List(ImgListRaw).get(n)).select('VV')                                        # Selects the next image in the list
     year = ee.Image(img).getString('system:index').slice(-50, 21)                                  # Select out the year
     month = ee.Image(img).getString('system:index').slice(-46, 23)                                 # Select out the month
     day = ee.Image(img).getString('system:index').slice(-44, 25)                                   # Select out the day
     NxtDate = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                # Format to date format e.g. 2023_01_01
     NxtDate = NxtDate.getInfo()                                                                    # ee.String an object on server side so need to extract it
     with output_widget:                                                                            # Method to update date widget
      print(NxtDate)                                                                                # Updates the image date widget
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     empty = ee.Image().byte()                                                                      # Empty image
     outline = empty.paint(featureCollection= Oil_Polygons,color= 'Colour')                         # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map

 PrevButton.on_click(on_button_clicked)                                                             # Action the previous image button
 return(MultiMap)                                                                                   # Display the map



In [ ]:
def SetUp():                                                                                        # Function for drop down menu
  global WRKdropdown, StartDate, EndDate, orbit, SatelliteNo, max_cloud_cover, Dilation, PixelFilter                                                     # Need this as otherwise it looks internally of the function for values
  Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                          # Load in the wreck data
  WRKdropdown = widgets.Dropdown(options=list(Wrecks.Wreck_ID),                                     # List of the wrecks from our data frame
                            value='HMS REPULSE',                                                    # Default value, here the repulse as one using as an example
                            description='Wreck:')                                                   # Descriptor in front of dropdown
  StartDate = widgets.DatePicker(description='Start Date')                                          # Setup the calendar to pick the start date to investigate
  EndDate = widgets.DatePicker(description='End Date')                                              # Setup the calendar to pick the end date to investigate
  orbit = widgets.RadioButtons(options=list(['ASCENDING','DESCENDING','BOTH']),                                # List of the wrecks from our data frame
                            value='DESCENDING',                                                     # Default value, here the repulse as one using as an example
                            description='Sentinel-1 Orbit Pass:')                                              # Descriptor in front of dropdown
  SatelliteNo = widgets.RadioButtons(options=list(['SENTINEL-1','SENTINEL-2', 'BOTH']),                                # List of the wrecks from our data frame
                            value='BOTH',                                                     # Default value, here the repulse as one using as an example
                            description='Satellites:')                                              # Descriptor in front of dropdown
  max_cloud_cover = widgets.IntSlider(value = 30,
                                      min = 0,
                                      max = 100,
                                      step = 1,
                                      description = 'Max cloud%',
                                      readout = True,
                                      orientation = 'horizontal',
                                      layout=widgets.Layout(width='400px'))
  Dilation = widgets.IntSlider(value = 3,
                                      min = 0,
                                      max = 30,
                                      step = 1,
                                      description = 'Dilation factor',
                                      readout = True,
                                      orientation = 'horizontal',
                                      layout=widgets.Layout(width='400px'))
  PixelFilter = widgets.IntSlider(value = 250,
                                      min = 0,
                                      max = 1000,
                                      step = 1,
                                      description = 'Pixel Filter',
                                      readout = True,
                                      orientation = 'horizontal',
                                      layout=widgets.Layout(width='400px'))
  return(display(WRKdropdown,StartDate,EndDate, SatelliteNo,orbit, max_cloud_cover, Dilation, PixelFilter))

# 3 Run the tool

##3.1 Selecting the desired parameters

Use the widgets to select the wreck and dates of interest.

Change the orbit pass if the wreck is at the edge of an tile or doesnt show
Sentinel- Level1 C has oil for HMS REPULSE AND PRINCE OF WALES on 17/03/2016 in a lovely image

War Mahtar image 9 december 2021

# Combining Sentinel-1 and sentinel-2

In [ ]:
def addDate(feature):                                                                               # Function to add date to the individual polygons
  return feature.set({'Date': date_string,'Colour': n})                                                 # Adds the date to the feature part of each polygon in the feature collection

def add_S1Gamma_date(image):
 year = ee.Image(image).getString('system:index').slice(-50, 21)                                      # Select out the year
 month = ee.Image(image).getString('system:index').slice(-46, 23)                                     # Select out the month
 day = ee.Image(image).getString('system:index').slice(-44, 25)                                       # Select out the day
 Date = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                    # Format to date format e.g. 2023_01_01
 Satellite = ee.Image(image).getString('system:index').slice(0, 3)
 return image.set('Date', Date, 'Satellite',Satellite)

def add_S1_date(image):
    date = image.date().format('YYYY-MM-dd')
    Satellite = ee.Image(image).getString('familyName')
    platform_number = ee.Image(image).getString('platform_number')
    Satellite = Satellite.cat(platform_number)
    return image.set('Date', date, 'Satellite',Satellite )

def add_S2_date(image):
    date = image.date().format('YYYY-MM-dd')
    Satellite = ee.Image(image).getString('SPACECRAFT_NAME')
    return image.set('Date', date, 'Satellite',Satellite )

def update_slider(img):
  if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
       Slider_value=[-30,-23.5]                                                                                   # Default
       Slider_min=-30.0                                                                                     # Min value
       Slider_max=-15.0                                                                                     # Max value
       Slider_step=0.5
       ChangeThreshformat='.1f'
  else:
       Slider_value=[2350,3000]                                                                                   # Default
       Slider_min=1500                                                                                  # Min value
       Slider_max=7000                                                                                    # Max value
       Slider_step=25
       ChangeThreshformat='d'
  ChangeThresh.max = Slider_max
  ChangeThresh.min = Slider_min
  ChangeThresh.step = Slider_step
  ChangeThresh.value = Slider_value
  ChangeThresh.readout_format=ChangeThreshformat
def OilMapping():                                                                                   # Function to display map
 global ImgList, Oil_Polygons, Datatable, img, n, date_string, output_widget, ChangeThreshWidget, ChangeThresh, ImgListGM, MultiMap, img_B_OSI                        # Need to make sure it brings the data from outside the function
 Oil_Polygons = ee.FeatureCollection([])                                                            # Need an empty FeatureCollection to Append our polygons to, add here so resets
 Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                           # Load in the wreck list
 Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Latitude'])               # Get the latitude of selected wreck
 Lon = Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Longitude']                             # Get the longitude of selected wreck
 geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                 # Loction of chosen wreck


 if SatelliteNo.value == 'SENTINEL-1' or SatelliteNo.value == 'BOTH':
 # Get the Sentinel-1 image collection
    S1ImgCol = (ee.ImageCollection('COPERNICUS/S1_GRD').                                               # Selects the Sentinel 1 image collection
      filterDate(str(StartDate.value), str(EndDate.value)).                                           # Selects only the dates from time period chosen above
      filterMetadata('instrumentMode', 'equals', 'IW').                                               # Selects the instrument mode that we want
      filter(ee.Filter.eq('orbitProperties_pass', str(orbit.value))).                                 # Selects the orbit path we want
      filterBounds(geom))                                                                             # Selects only images that our wreck is contained within
    S1ImgCol = S1ImgCol.map(add_S1_date)
    ImgCol_gammaMap = S1ImgCol.map(gammaMap)                                                             # Process the ImageCollection through the gammaMap algorithm





 if SatelliteNo.value == 'SENTINEL-2' or SatelliteNo.value == 'BOTH':
# Get the sentinel-2 image collection
  S2ImgCol = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED').                                         # Selects the Sentinel 1 image collection
     filterDate(str(StartDate.value), str(EndDate.value)).                                           # Selects only the dates from time period chosen above
     #filterMetadata('MGRS_TILE', 'equals', '32SQG').
     filterBounds(geom).                                                                             # Selects only images that our wreck is contained within
     filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', max_cloud_cover.value))
  S2ImgCol = S2ImgCol.map(add_S2_date)


 if SatelliteNo.value == 'BOTH':
 ## Image collection with NO S1 gamma correction
   ImageCol = S1ImgCol.merge(S2ImgCol)
   ImageCol = ImageCol.sort("system:time_start")
   ImgList = ee.ImageCollection(ImageCol).toList(99999)                                               # Creates a list of the d images to select from
   ## Image collection WITH S1 gamma correction
   ImageColGM = ImgCol_gammaMap.merge(S2ImgCol)
   ImageColGM = ImageColGM.sort("system:time_start")
   ImgListGM = ee.ImageCollection(ImageColGM).toList(99999)                                               # Creates a list of the d images to select from
 elif SatelliteNo.value == 'SENTINEL-1':
   ImageCol = S1ImgCol
   ImgListGM = ee.ImageCollection(ImgCol_gammaMap).toList(99999)                                               # Creates a list of the d images to select from
 elif  SatelliteNo.value == 'SENTINEL-2':
     ImageCol = S2ImgCol
 ImgList = ee.ImageCollection(ImageCol).toList(99999)                                               # Creates a list of the d images to select from
 Datatable = pd.DataFrame(columns = ['Wreck_Name', 'Date', 'Oil_Area_m2','Low_Threshold','High_Threshold', 'Satellite','Comment'])              # Empty dataframe for polygon oil spill area
 MultiMap = geemap.Map()                                                                            # Base map
 MultiMap.centerObject(geom, 10)                                                                    # Center the map on the wreck
 MultiMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                   # Add wreck locations

 n = 0                                                                                              # used to add or subtract to change the image
 img = ee.Image(ee.List(ImgList).get(n))
 if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
  img = ee.Image(ee.List(ImgList).get(n)).select('VV')
  img_params = {'bands':'VV', 'min':-25, 'max':5}                                                    # Display setting for the VV band
 else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}
 IMgdate = img.date()                                                                                  # Get the acquisition date of the image
 date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                                  # Format the date as a string
 MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                         # Add the image to the map

 output_widget = widgets.Output(layout={'border': '1px solid black'})                               # Set up widget for adding date
 output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')            # Method to add widget
 MultiMap.add_control(output_control)                                                               # Adds the widget
 with output_widget:                                                                                # The date widget update
   print(date_string)


 if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
       Slider_value=[-30,-23.5]                                                                                   # Default
       Slider_min=-30.0                                                                                     # Min value
       Slider_max=-15.0                                                                                     # Max value
       Slider_step=0.5
       ChangeThreshformat='.1f'
 else:
       Slider_value=[2350,3000]                                                                                   # Default
       Slider_min=1500                                                                                     # Min value
       Slider_max=7000                                                                                     # Max value
       Slider_step=25
       ChangeThreshformat='d'

 ChangeThresh =  widgets.FloatRangeSlider(                                                               # Defines the Oil threshold slider
       value=Slider_value,                                                                                   # Default
       min=Slider_min,                                                                                     # Min value
       max=Slider_max,                                                                                     # Max value
       step=Slider_step,                                                                                      # interval step value
       description='Thresh:',                                                       # Description
       orientation='horizontal',                                                                      # Horizontal or Vertical
       readout=True,                                                                                  # Prints value
       readout_format=ChangeThreshformat,
       layout=widgets.Layout(width='400px'))                                                              # Format of value (1 decimal here)
 ChangeThreshOutput = widgets.Output()                                                              # Output to display for the widget
 ChangeThreshWidget = ipyleaflet.WidgetControl(widget=ChangeThresh, position='bottomleft')          # Method to add to the map


 MultiMap.add_control(ChangeThreshWidget)
#################################################################################
 DetectOilButton = widgets.Button(description="Detect potential oil")                               # Button widget to add to the map
 DetectOilButtonOutput = widgets.Output()                                                           # Output to display for the widget
 DetectOilButtonWidget = ipyleaflet.WidgetControl(widget=DetectOilButton, position='bottomright')   # Method to add to the map
 MultiMap.add_control(DetectOilButtonWidget)                                                        # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
   global Oil_Polygonstmp, n, ChangeThreshWidget, ChangeThresh, ImgListGM, img_B_OSI                                                                     # Need to make sure it brings the data from outside the function
   #ChangeThreshWidget.clear_output()                                                                     # Remove display of the date of last image
   with DetectOilButtonOutput:                                                                      # Below happens when button clicked
     img = ee.Image(ee.List(ImgList).get(n))
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
       img = ee.Image(ee.List(ImgList).get(n)).select('VV')
       mask_lt = img.lt(ChangeThresh.value[1])
       mask_gt = img.gt(ChangeThresh.value[0])
       mask = mask_gt.And(mask_lt)
       Oil = img.updateMask(mask).toInt().rename('Oil')
     else:
       rB = img.expression(
          '(b("B3") + b("B2"))',
          {'b': img.select(["B3", "B2"])}).rename('rB')
       gB = img.expression(
         '(b("B3") + b("B4")) / b("B2")',
         {'b': img.select(["B3", "B4", "B2"])}).rename('gB')
       bB = img.expression(
         '(b("B6") + b("B7")) / b("B5")',
         {'b': img.select(["B6", "B7", "B5"])}).rename('bB')
       img_B_OSI = ee.Image([rB, gB, bB])
       img_params_B_OSI = {'min': 0,'max': 2,'bands': ['rB','gB','bB']}
       vis_params = {'min': 0,'max': 5,'Bands': ['Oil','Oil2','Oil3']}
       Oil = img_B_OSI.select('rB').gt(ChangeThresh.value[0]).And(img_B_OSI.select('rB').lt(ChangeThresh.value[1])).selfMask()
     #OilDIL = Oil.focal_max(kernel=ee.Kernel.circle(radius=Dilation.value))
     AoI = ee.FeatureCollection(MultiMap.draw_features)                                              # Extracts the drawn polygon from the map
     MultiMap.addLayer(Oil, {'palette': 'FF0000'}, ('Pixels'))                                       # Displays those with pixels value 23.5
     Oil_Polygonstmp = Oil.reduceToVectors(geometry = AoI,                                           # Polygon extracted from the map
                             scale = 10,                                                            # Resolution of data in meters
                             geometryInNativeProjection =False,                                      # Use the image projection
                             maxPixels = 1e10,                                                      # Max pixels GEE will deal with
                             eightConnected =True)                                                  # Polgons will count as connect if at a diagonal
     Oil_Polygonstmp = Oil_Polygonstmp.filter(ee.Filter.gt('count',PixelFilter.value))                              # Filters the polgons whic have more than 25 pixels
     Oil_Polygonstmp = Oil_Polygonstmp.map(addDate)                                                  # Adds image date to the polygons
     empty = ee.Image().byte()
     outline = empty.paint(featureCollection= Oil_Polygonstmp,
              color= 'Colour')
     MultiMap.addLayer(outline.randomVisualizer(),name ='Temp Oil Spill Polygons')

 DetectOilButton.on_click(on_button_clicked)                                                        # Makes changes when clicked
########################################################################################################
 NextButton = widgets.Button(description="Next image")                                              # Button widget to add to the map
 NextButtonOutput = widgets.Output()                                                                # Output to display for the widget
 NextButtonWidget = ipyleaflet.WidgetControl(widget=NextButton, position='bottomright')             # Method to add to the map
 MultiMap.add_control(NextButtonWidget)                                                             # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
   global n, date_string, img, ImgList, Datatable, Oil_Polygons, Oil_Polygonstmp, output_widget     # Need this as otherwise it looks internally of the function for values
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   empty = ee.Image().byte()                                                                        # Empty image
   Sat = ee.Image(img).getString('Satellite').getInfo()
   outline = empty.paint(featureCollection= Oil_Polygons, color= 'Colour')                          # Add the polygon and colour
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value],
                       'Date':[date_string],
                       'Oil_Area_m2': [spillarea],
                       'Low_Threshold':[ChangeThresh.value[0]],
                       'High_Threshold':[ChangeThresh.value[1]],
                       'Satellite':[Sat],
                       'Comment':''})# Combine the data together
   if date_string in Datatable['Date'].values:                                                          # Check if the date already exists in the DataFrame
    # Overwrite the existing row if already exist and changing the thresholds
    Datatable.loc[Datatable['Date'] == date_string, ['Wreck_Name', 'Date',
                                                     'Oil_Area_m2','Low_Threshold',
                                                     'High_Threshold','Satellite','Comment' ]] = [WRKdropdown.value, date_string, spillarea, ChangeThresh.value[0],ChangeThresh.value[1],Sat,'']
   else:                                                                                            # Otherwise just append
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n += 1                                                                                           # minus one to the number in the image list
   with NextButtonOutput:                                                                           # Below happens when button clicked
     PrevDate = date_string
     img = ee.Image(ee.List(ImgList).get(n))
     IMgdate = img.date()                                                                                  # Get the acquisition date of the image
     date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                                  # Format the date as a string
     if date_string == PrevDate:
        n+=1
        img = ee.Image(ee.List(ImgList).get(n))
        IMgdate = img.date()                                                                                  # Get the acquisition date of the image
        date_string = IMgdate.format('YYYY-MM-dd').getInfo()
     if Datatable['Date'].isin([date_string]).any():
        Oil_Polygons =  Oil_Polygons.filter(ee.Filter.neq('Date', date_string))


     update_slider(img)
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
       img = ee.Image(ee.List(ImgList).get(n)).select('VV')
       img_params = {'bands':'VV', 'min':-25, 'max':5}                                                    # Display setting for the VV band
     else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}


     output_widget = widgets.Output(layout={'border': '1px solid black'})                               # Set up widget for adding date
     output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')            # Method to add widget
     MultiMap.add_control(output_control)                                                               # Adds the widget
     with output_widget:                                                                                # The date widget update
       print(date_string)
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     empty = ee.Image().byte()                                                                      # Empty image
     outline = empty.paint(featureCollection= Oil_Polygons,color= 'Colour')                         # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map
 NextButton.on_click(on_button_clicked)                                                             # actions the function when the button clicked

############################################################################################################
 PrevButton = widgets.Button(description="Previous image")                                          # Button widget to add to the map
 PrevButtonOutput = widgets.Output()                                                                # Output to display for the widget
 PrevButtonWidget = ipyleaflet.WidgetControl(widget=PrevButton, position='bottomleft')              # Method to add to the map
 MultiMap.add_control(PrevButtonWidget)                                                             # Adds to the map

 def on_button_clicked(b):                                                                          # Define a function for what happens on button click
  ## Operations that need to happen before the button action
   global n, date_string, img, ImgListRaw, Datatable, Oil_Polygons, Oil_Polygonstmp, output_widget      # Need this as otherwise it looks internally of the function for values
   if Datatable['Date'].isin([date_string]).any():
        Oil_Polygons = Oil_Polygons.filter(ee.Filter.neq('Date', date_string))
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   Sat = ee.Image(img).getString('Satellite').getInfo()
   empty = ee.Image().byte()                                                                        # Empty image
   outline = empty.paint(featureCollection= Oil_Polygons, color= 'Colour')                          # Add the polygon and colour
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value],
                       'Date':[date_string],
                       'Oil_Area_m2': [spillarea],
                       'Low_Threshold':[ChangeThresh.value[0]],
                       'High_Threshold':[ChangeThresh.value[1]],
                       'Satellite':[Sat],
                       'Comment':''})# Combine the data together

   if date_string in Datatable['Date'].values:                                                          # Check if the date already exists in the DataFrame
    # Overwrite the existing row if already exist and changing the thresholds
    Datatable.loc[Datatable['Date'] == date_string, ['Wreck_Name', 'Date',
                                                     'Oil_Area_m2','Low_Threshold',
                                                     'High_Threshold','Satellite','Comment' ]] = [WRKdropdown.value, date_string, spillarea, ChangeThresh.value[0],ChangeThresh.value[1],Sat,'']
   else:                                                                                            # Otherwise just append
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n -= 1                                                                                           # minus one to the number in the image list
   with PrevButtonOutput:                                                                           # Below happens when button clicked
     img = ee.Image(ee.List(ImgList).get(n))
     update_slider(img)
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
       img = ee.Image(ee.List(ImgList).get(n)).select('VV')
       img_params = {'bands':'VV', 'min':-25, 'max':5}                                                    # Display setting for the VV band
     else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}
     IMgdate = img.date()                                                                                  # Get the acquisition date of the image
     date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                                  # Format the date as a string
     if Datatable['Date'].isin([date_string]).any():
        Oil_Polygons =  Oil_Polygons.filter(ee.Filter.neq('Date', date_string))
     output_widget = widgets.Output(layout={'border': '1px solid black'})                               # Set up widget for adding date
     output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')            # Method to add widget
     MultiMap.add_control(output_control)                                                               # Adds the widget
     with output_widget:                                                                                # The date widget update
       print(date_string)
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     empty = ee.Image().byte()                                                                      # Empty image
     outline = empty.paint(featureCollection= Oil_Polygons,color= 'Colour')                         # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map

 PrevButton.on_click(on_button_clicked)


 return MultiMap


In [ ]:
def value_detector(img):
  Line = ee.FeatureCollection(MultiMap.draw_features)
  if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B':
   img2p = ee.Image(ee.List(ImgListGM).get(n)).select('VV')
   LineGraph = img2p.sample(Line, 10).aggregate_array("VV").getInfo()
   LineGraphDF = pd.DataFrame(LineGraph, columns=['VV'])
   fig = px.line(data_frame=LineGraphDF,y = 'VV', markers = True, width=1000, height=800).update_layout(
                 xaxis_title="Pixel number along the line", yaxis_title="band VV")
  else:
   rB = img.expression(
          '(b("B3") + b("B2"))',
          {'b': img.select(["B3", "B2"])}).rename('rB')
   gB = img.expression(
         '(b("B3") + b("B4")) / b("B2")',
         {'b': img.select(["B3", "B4", "B2"])}).rename('gB')
   bB = img.expression(
         '(b("B6") + b("B7")) / b("B5")',
         {'b': img.select(["B6", "B7", "B5"])}).rename('bB')
   img_B_OSI = ee.Image([rB, gB, bB])
   B_OSI_rB = img_B_OSI.sample(Line, 10).aggregate_array("rB").getInfo()
   B_OSI_rBDF = pd.DataFrame(B_OSI_rB, columns=['rB'])
   fig = px.line(data_frame=B_OSI_rBDF,y = 'rB', markers = True, width=1000, height=800).update_layout(
                 xaxis_title="Pixel number along the line", yaxis_title="bands B3 + B2")
  return fig.show()


In [ ]:
def Display_Data():
  global Comment, CommitButOutput, CommitBut
  CommitButOutput = widgets.Output()
  Comment= widgets.Textarea(
      placeholder='Add comment for image here',
     description='Comment:',
     disabled=False)
  CommitBut = widgets.Button(
     value=False,
     description='Commit comment',
     disabled=False,
     button_style='', # 'success', 'info', 'warning', 'danger' or ''
     tooltip='Description',
     icon='check' # (FontAwesome names without the `fa-` prefix)
)
  def on_combutton_clicked(b):                                                                          # Define a function for what happens on button click
    global Datatable, n,comment                                                            # Need to make sure it brings the data from outside the function
    with CommitButOutput:                                                                      # Below happens when button clicked
      Row = n-1
      Datatable.iloc[Row,6] = Comment.value
  CommitBut.on_click(on_combutton_clicked)
  display(Comment,CommitBut,Datatable)